Staples Project: Price Discrimination

$$π(P_i,X_i)=[B_0+B_1* X_i+B_2Pi+B_3* X_iP_i](P_i-C_i)$$
$$\frac{2π}{2P_i}= (B_2+B_3*X_i)(P_i-C_i)+(B_0+B_1* X_i +B_2P_i+B_3*X_iP_i)$$
$$P_i^M=\frac{(B_2+ B_3 * X_i)Ci - B_2-B_1*X_i}{2(B_2+B_3 * X_i)}$$


In [31]:
import pandas as pd
from linearmodels import OLS
import numpy as np

In [32]:
# Import data
df_data = pd.read_csv("StaplesData.csv", index_col=0)

# Set parameters
c = 10 # Marginal cost

df_data.head()

,Purchase,Price,NotNearCompetingStore,WealthyZipCode,Weekday,ExistingCustomer
1,1,13,0,0,1,0
2,1,13,1,0,1,1
3,1,13,0,0,1,0
4,1,13,0,0,1,0
5,0,15,0,0,1,0


In [33]:
covariates = ['NotNearCompetingStore', 'WealthyZipCode', 'Weekday', 'ExistingCustomer']

regression_formula =  'Purchase ~ 1 + Price * ({})'.format(' + '.join(covariates))
print('regression formula', regression_formula, '\n')

model = OLS.from_formula(regression_formula, df_data)

res = model.fit()

print(res.summary)

# storing the coefficients as vectors 
beta_0 = res.params.Intercept
beta_1 = res.params[covariates]
beta_2 = res.params.Price
beta_3 = res.params[['Price:' + cov for cov in covariates]]

regression formula Purchase ~ 1 + Price * (NotNearCompetingStore + WealthyZipCode + Weekday + ExistingCustomer) 

                            OLS Estimation Summary                            
Dep. Variable:               Purchase   R-squared:                      0.3230
Estimator:                        OLS   Adj. R-squared:                 0.3228
No. Observations:               40000   F-statistic:                 6.511e+04
Date:                Thu, Jan 27 2022   P-value (F-stat)                0.0000
Time:                        00:52:47   Distribution:                  chi2(9)
Cov. Estimator:                robust                                         
                                                                              
                                      Parameter Estimates                                      
                             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------

Intercept: 0.0000 
Price: 0.0000
NotNearCompetingStore: 0.0000
WealthyZipCode: 0.0192
Weekday: 0.0628 
Price:NotNearCompetingStore: 0.0000 
Price:WealthyZipCode: 0.0000
These are all coefficient estimates whose p-values are below 10%

In [34]:
# Not being near a competing store
print( 1- (beta_2 + beta_3['Price:NotNearCompetingStore']) / beta_2 ) # reduces by X percent
print(beta_2) # from X percentage points
print(beta_2 + beta_3['Price:NotNearCompetingStore']) # to X percentage points

0.37556495351773755
-0.15857552452320436
-0.09902011502659627


In [35]:
# Wealthy zip code
print( 1- (beta_2 + beta_3['Price:WealthyZipCode']) / beta_2 ) # reduces by X percent
print(beta_2) # from X percentage points
print(beta_2 + beta_3['Price:WealthyZipCode']) # to X percentage points

0.2954305570981153
-0.15857552452320436
-0.11172746897118825


conditional on the other covariates(WealthyZipCode, Weekday, and ExistingCustomer) , not being near a competing store reduces price responsiveness by 0.37556495351773755 percentage points per dollar of price increase.

conditional on the other covariates(NotNearCompetingStore, Weekday, and ExistingCustomer) , being in a wealthy zip code reduces price responsiveness by 0.2954305570981153 percentage points per dollar of price increase.

The signs and the magnitudes are consistent with what I expected.

In [36]:
# Profit maximizing prices
df_data['monopoly_price'] = c / 2 - (beta_0 + np.dot(df_data[covariates], beta_1)) / (
    2 * (beta_2 + np.dot(df_data[covariates], beta_3)))
df_data.monopoly_price.describe()

count    40000.000000
mean        13.520190
std          1.276742
min         12.122566
25%         13.104497
50%         13.230974
75%         13.455927
max         17.598529
Name: monopoly_price, dtype: float64

min:12.122566
max:17.598529
mean:13.520190

In [37]:
# Expected profit per consumer

# Computing purchase probabilities
df_data['purchase_proba'] = beta_0 + np.dot(df_data[covariates], beta_1) + beta_2 * df_data.monopoly_price +  np.dot(df_data[covariates], beta_3) * df_data.monopoly_price

# Inserting the purchase probabilities and the prices into the profit function
df_data['profit'] = (df_data.purchase_proba * (df_data.Price - c))

print(df_data[['purchase_proba', 'profit']].describe())

       purchase_proba        profit
count    40000.000000  40000.000000
mean         0.472592      1.892363
std          0.142677      0.757085
min          0.210177      0.630530
25%          0.367183      1.449082
50%          0.528805      1.586415
75%          0.528805      2.644025
max          0.728943      3.644714


min:0.630530
max:3.644714
mean:1.892363

$$π(P)=[\alpha_0+\alpha_2 * P](P-C)$$
take FOC:
$$P^unif=\frac{C}{2} - \frac{\alpha_0}{2*\alpha_2} $$

In [38]:
# Estimate average demand function

model = OLS.from_formula('Purchase ~ 1 + Price', df_data)
res = model.fit()
print(res.summary)

# Storing the coefficients
beta_0h = res.params.Intercept
beta_2h = res.params.Price

                            OLS Estimation Summary                            
Dep. Variable:               Purchase   R-squared:                      0.0777
Estimator:                        OLS   Adj. R-squared:                 0.0777
No. Observations:               40000   F-statistic:                    3367.6
Date:                Thu, Jan 27 2022   P-value (F-stat)                0.0000
Time:                        00:52:48   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      2.2986     0.0332     69.155     0.0000      2.2335      2.3638
Price         -0.1361     0.0023    -58.031     0.00

$$\alpha_0 =2.2986$$
$$\alpha_2 =-0.1361$$

A dollar increase in price is associated with a 0.1361 decline in purchase probability.

In [39]:
# Profit-maximizing uniform price 
uniform_monopoly_price = c / 2 - beta_0h / (2 * beta_2h)
print(uniform_monopoly_price) 

13.443140640542051


The profit-maximizing uniform price is $13.44.

In [40]:
# Expected profit under uniform price
df_data['purchase_proba_unif_price'] = beta_0 + np.dot(df_data[covariates], beta_1) + \
    (beta_2 + np.dot(df_data[covariates], beta_3)) * uniform_monopoly_price


#Inserting the purchase probabilities and the prices into the profit function
df_data['profit_unif_price'] = (df_data.purchase_proba_unif_price * (uniform_monopoly_price - c))

print(df_data[['purchase_proba_unif_price', 'profit_unif_price']].describe())

       purchase_proba_unif_price  profit_unif_price
count               40000.000000       40000.000000
mean                    0.469590           1.616865
std                     0.242354           0.834458
min                     0.079413           0.273431
25%                     0.438597           1.510150
50%                     0.494080           1.701188
75%                     0.554729           1.910011
max                     1.049369           3.613125


In [41]:
# Profit loss from uniform pricing

Profit_loss_uniform = sum(df_data['profit'] - df_data['profit_unif_price'])
print(Profit_loss_uniform)

11019.925474385796


The expected profit loss from uniform pricing instead of price discrimination is $11019.93

Examples of data other than the variables in StaplesData.csv that might better predict purchase probability are whether there is a staples nearby their IP address and how many orders they have made with staples previously.

Some caveats I would like my manager to understand is that; there is a possibility that some low-income people live in zipcode areas that are considered 'wealthy' and being an existing customer may or may not be a good thing depending on the persons past experience with a staples product.